## Import Section

In [ ]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (EarlyStopping, ModelCheckpoint, 
                                        ReduceLROnPlateau)
np.random.seed(452)

## Read image data 

In [ ]:
main_dir = '../input/cassava-leaf-disease-classification'
train_df = pd.read_csv(main_dir + '/train.csv')
train_df['label'] = train_df['label'].astype('str')
train_df.head()

In [ ]:
# check the shape of the images
train_image_paths = main_dir + '/train_images'

for file in train_df.image_id[:5]:
    print(plt.imread(os.path.join(train_image_paths, file)).shape)

## Set up some global variables

In [ ]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16 
STEPS_PER_EPOCH = len(train_df) * 0.8 // BATCH_SIZE
VALIDATION_STEPS = len(train_df) * 0.2 // BATCH_SIZE
EPOCHS = 10

## Data Augmentation

In [ ]:
# create train and validation augmentations
train_aug = ImageDataGenerator(rotation_range = 40,
                               width_shift_range = 0.2, 
                               height_shift_range = 0.2, 
                               zoom_range = 0.2,
                               shear_range = 0.2, 
                               brightness_range = [0.2, 1.0], 
                               horizontal_flip = True, 
                               vertical_flip = True, 
                               validation_split = 0.2, 
                               fill_mode = 'nearest')


val_aug = ImageDataGenerator(validation_split = 0.2)


# create train and validation generators
# If class_mode = 'sparse', y_col = 'label' column values must be strings
train_gen = train_aug.flow_from_dataframe(train_df, 
                                          directory = train_image_paths, 
                                          subset = 'training', 
                                          x_col = 'image_id',
                                          y_col = 'label', 
                                          target_size = IMAGE_SIZE, 
                                          batch_size = BATCH_SIZE, 
                                          class_mode = 'sparse', 
                                          seed = 42, 
                                          shuffle = True)


val_gen = val_aug.flow_from_dataframe(train_df, 
                                      directory = train_image_paths, 
                                      subset = 'validation', 
                                      x_col = 'image_id',
                                      y_col = 'label', 
                                      target_size = IMAGE_SIZE, 
                                      batch_size = BATCH_SIZE, 
                                      class_mode = 'sparse', 
                                      seed = 42, 
                                      shuffle = False)

## Load pretrained EfficientNetB7 Model

In [ ]:
# base model
base_model = tf.keras.applications.EfficientNetB0(weights = 'imagenet', 
                                                  include_top = False, 
                                                  input_shape = IMAGE_SIZE + (3, ))

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dropout(0.5)(x)
output = layers.Dense(5, activation = 'softmax')(x)
model = tf.keras.Model(base_model.input, output)

## Callbacks

In [ ]:
# save model weights
model_path = ModelCheckpoint('best_weights.h5', 
                             save_best_only = True, 
                             monitor = 'val_loss', 
                             mode = 'min', 
                             verbose = 1)

# learning rate scheduler
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', 
                              factor = 0.3, 
                              patience = 2, 
                              min_lr = 1e-6, 
                              mode = 'min', 
                              verbose = 1)

# early stopping
early_stopping = EarlyStopping(monitor = 'val_loss', 
                               patience = 3, 
                               mode = 'min', 
                               verbose = 1, 
                               restore_best_weights = True)

## Compile and fit the model

In [ ]:
# compile the model
model.compile(optimizer = tf.keras.optimizers.Adam(1e-3), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['accuracy'])

# fit the model
history = model.fit(train_gen, 
                    steps_per_epoch = STEPS_PER_EPOCH, 
                    epochs = EPOCHS, 
                    validation_data = val_gen, 
                    validation_steps = VALIDATION_STEPS, 
                    callbacks = [model_path, early_stopping, reduce_lr])

In [ ]:
# plot the loss and accuracy of the model
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
history_df.loc[:, ['accuracy', 'val_accuracy']].plot();